In [19]:
# ssh -L 9000:localhost:9000 thanrada@10.121.101.110
# http://127.0.0.1:9000/?token=3fa074da2984241394d534e672b246eff7b82102c9b6f317
# https://iservice.dopa.go.th/web/map/license/map/

In [57]:
import pandas as pd
import numpy as np

In [58]:
hotel = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/bora/hotel_bangkok.csv').iloc[:,[0,1,3,4]]
hotel.columns = ['Hotel_Ref','Hotel_Name','Hotel_Address','N']
hotel["Hotel_Address"]= hotel["Hotel_Address"].replace("บ้านเลขที่ ","", regex=True)
hotel.head()

,Hotel_Ref,Hotel_Name,Hotel_Address,N
0,101/2563,โรงแรม สุทธิสารพ้อยส์,45 ซ.ยิ้มอุปถัมภ์ แขวงรัชดาภิเษก เขตดินแดง จ.ก...,42
1,136/2563,โรงแรม คณาบูทีค 2,"3,5,7,9,11 ซ.จรัญสนิทวงศ์ 57/2 แขวงบางบำหรุ เข...",24
2,152/2563,วัลย์ลดา เพลส,71/1 ซ.รามคำแหง 53 (จันทร์ศรีชวาลา) แขวงพลับพล...,50
3,214/2563,โรงแรม นิวสยาม พาเลซวิลล์ 2,56 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48
4,215/2563,โรงแรม นิวสยาม พาเลซวิลล์ 3,58 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48


In [59]:
trip_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/OTA/trip_com_bkk.csv')[['name','name href','address']].dropna()
trip_com['OTA'] = 'trip_com'

booking_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/main/OTA/booking_com_bkk_with_addr.csv', engine='python')[['name href','name','address']].dropna()
booking_com['OTA'] = 'booking_com'

# รวมสอง DataFrame เข้าด้วยกัน
# OTA_df = pd.concat([trip_com, booking_com], ignore_index=True)
OTA_df = pd.concat([trip_com], ignore_index=True).sample(500,random_state=42)
OTA_df.head()

,name,name href,address,OTA
31,โรงแรมคิวว์ สุขุมวิท,https://th.trip.com/hotels/detail/?cityId=359&...,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",trip_com
413,โรงแรมสำราญเพลส,https://th.trip.com/hotels/detail/?cityId=359&...,"302 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
536,ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,https://th.trip.com/hotels/detail/?cityId=359&...,"27 ซ.สุขุมวิท 4 คลองเตย, เขตคลองเตย, กรุงเทพฯ,...",trip_com
745,ทู ทรี อะ โฮมลี่ โฮเทล,https://th.trip.com/hotels/detail/?cityId=359&...,"100 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ, เขตวัฒนา,...",trip_com
877,โรงแรมกลับ,https://th.trip.com/hotels/detail/?cityId=359&...,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com


## Method 1: Semantic search

### mrp/simcse-model-m-bert-thai-cased

In [60]:
# ---------- ตั้งค่า (from 614a1f16) ----------
import re, unicodedata
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1) Clean: ชื่อ/ที่อยู่ + สร้าง "คีย์" name+address
# =========================
def normalize_name(s: str) -> str:
    """ตัดคำ 'โรงแรม/Hotel' และ 'กรุงเทพ/กรุงเทพฯ/กรุงเทพมหานคร' + ลบเว้นวรรค/สัญลักษณ์"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel| โฮเทล|โฮเทล)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)  # ตัดคำ 'กรุงเทพ' ออกจาก 'ชื่อ'
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)  # ลบช่องว่างทั้งหมด
    return s

def normalize_addr(s: str) -> str:
    """คงโครงสร้างที่อยู่ไว้ (ไม่ลบช่องว่างทั้งหมด) แต่ normalize ให้เท่า ๆ กัน"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()

    # ทำให้คำหลักสอดคล้องกัน
    rep = {
        "กทม.": "กรุงเทพมหานคร",
        "กรุงเทพฯ": "กรุงเทพมหานคร",
        "จ.": "",              # จ.กรุงเทพมหานคร -> กรุงเทพมหานคร
        "เขต ": "",
        "แขวง ": "",
        "ถ.": "",
        "ถนน ": "",        # กันกรณีมีช่องว่างเพี้ยน
        "บ้านเลขที่": "",
    }
    for k,v in rep.items():
        s = s.replace(k, v)

    # เก็บตัวอักษร/ตัวเลข/เว้นวรรคและเครื่องหมาย / , - บางส่วน
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

# apply clean
OTA_df["Trip_Name_n"]    = OTA_df["name"].map(normalize_name)
OTA_df["Trip_Address_n"] = OTA_df["address"].map(normalize_addr)
hotel["Hotel_Name_n"]    = hotel["Hotel_Name"].map(normalize_name)
hotel["Hotel_Address_n"] = hotel["Hotel_Address"].map(normalize_addr)

# คีย์ค้นหา = name + address (ตามที่ต้องการ)
OTA_df["Trip_Key"]   = (OTA_df["Trip_Name_n"]   + " " + OTA_df["Trip_Address_n"]).str.strip()
hotel["Hotel_Key"] = (hotel["Hotel_Name_n"] + " " + hotel["Hotel_Address_n"]).str.strip()

In [61]:
OTA_df[OTA_df['Trip_Name_n'].str.contains('กลับ', na=False)]

,name,name href,address,OTA,Trip_Name_n,Trip_Address_n,Trip_Key
877,โรงแรมกลับ,https://th.trip.com/hotels/detail/?cityId=359&...,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,กลับ,22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400,กลับ 22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400


In [62]:
hotel[hotel['Hotel_Name'].str.contains('กลับ', na=False)]

,Hotel_Ref,Hotel_Name,Hotel_Address,N,Hotel_Name_n,Hotel_Address_n,Hotel_Key
450,141/2565,กลับโฮเทล,22/1 ซ.สมประสงค์ 2 แขวงถนนพญาไท เขตราชเทวี จ.ก...,79,กลับ,22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุงเทพม...,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...


In [63]:
# 0) ONE SOURCE OF TRUTH
OTA_df = OTA_df.reset_index(drop=True).copy()
hotel  = hotel.reset_index(drop=True).copy()

# 1) ensure columns exist & fillna
for col in ["Trip_Name_n","Trip_Address_n"]:
    if col not in OTA_df.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {col} ใน OTA_df — ต้อง normalize ให้เรียบร้อยก่อน")
for col in ["Hotel_Name_n","Hotel_Address_n"]:
    if col not in hotel.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {col} ใน hotel — ต้อง normalize ให้เรียบร้อยก่อน")

OTA_df["Trip_Name_n"]    = OTA_df["Trip_Name_n"].fillna("").astype(str)
OTA_df["Trip_Address_n"] = OTA_df["Trip_Address_n"].fillna("").astype(str)
hotel["Hotel_Name_n"]    = hotel["Hotel_Name_n"].fillna("").astype(str)
hotel["Hotel_Address_n"] = hotel["Hotel_Address_n"].fillna("").astype(str)

# 2) สร้างคีย์รวม (ถ้ายังไม่มี/อยากทับ)
OTA_df["Trip_Key"] = (OTA_df["Trip_Name_n"] + " " + OTA_df["Trip_Address_n"]).str.strip()
hotel["Hotel_Key"] = (hotel["Hotel_Name_n"] + " " + hotel["Hotel_Address_n"]).str.strip()

# 3) โหลดโมเดล (ถ้ายังไม่มี)
MODEL = "mrp/simcse-model-m-bert-thai-cased"
model = SentenceTransformer(MODEL)

# 4) Encode ใหม่ให้ “สอดคล้องกับ DataFrame นี้เท่านั้น”
emb_trip_name  = model.encode(OTA_df["Trip_Name_n"].tolist(),    convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_addr  = model.encode(OTA_df["Trip_Address_n"].tolist(),  convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_key   = model.encode(OTA_df["Trip_Key"].tolist(),        convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

emb_hotel_name = model.encode(hotel["Hotel_Name_n"].tolist(),     convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_addr = model.encode(hotel["Hotel_Address_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_key  = model.encode(hotel["Hotel_Key"].tolist(),         convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

# 5) ตรวจ alignment
assert emb_trip_key.shape[0]  == len(OTA_df), f"trip mismatch: emb={emb_trip_key.shape[0]} vs df={len(OTA_df)}"
assert emb_hotel_key.shape[0] == len(hotel),  f"hotel mismatch: emb={emb_hotel_key.shape[0]} vs df={len(hotel)}"


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [67]:
#Test
QUERY = "เดอะ คีย์ พรีเมียร์ สุขุมวิท"
TOPK  = 5
WEIGHTS = dict(key=0.2, name=0.6, addr=0.2)

# 1) เลือกแถว OTA ที่มีคำว่า 'กลับ'
mask = OTA_df["Trip_Name_n"].astype(str).str.contains(QUERY, na=False) | \
       OTA_df["name"].astype(str).str.contains(QUERY, na=False)
if not mask.any():
    raise ValueError(f"ไม่พบที่พักใน OTA_df ที่มีคำว่า '{QUERY}'")

trip_pos   = int(np.flatnonzero(mask)[0])
trip_row   = OTA_df.iloc[trip_pos]
trip_title = str(trip_row.get("name", f"OTA_idx_{trip_pos}"))
trip_addr  = str(trip_row.get("address", ""))

# 2) คำนวณความคล้ายกับฝั่ง hotel
sim_key  = cosine_similarity(emb_trip_key[trip_pos:trip_pos+1],  emb_hotel_key).ravel()
sim_name = cosine_similarity(emb_trip_name[trip_pos:trip_pos+1], emb_hotel_name).ravel()
sim_addr = cosine_similarity(emb_trip_addr[trip_pos:trip_pos+1], emb_hotel_addr).ravel()
score = WEIGHTS['key']*sim_key + WEIGHTS['name']*sim_name + WEIGHTS['addr']*sim_addr

# 3) เลือก Top-5 ตามคะแนนรวม
order = np.argsort(-score)[:TOPK]
result = hotel.iloc[order].copy()
result["sim_key"]  = sim_key[order]
result["sim_name"] = sim_name[order]
result["sim_addr"] = sim_addr[order]
result["score"]    = score[order]

# 4) แสดงผลลัพธ์พร้อมชื่อ OTA_df
cols = ["Hotel_Name", "Hotel_Address","Hotel_Key" ,"sim_key", "sim_name", "sim_addr", "score"]

print(f"📍 OTA ชื่อ: {trip_title}")
print(f"🏠 OTA ที่อยู่: {trip_addr}")
print(f"\nเพื่อนบ้านของ OTA นี้ (Top-{TOPK} โรงแรมที่ใกล้ที่สุด):")
display(result[cols].sort_values("score", ascending=False).reset_index(drop=True))


📍 OTA ชื่อ: เดอะ คีย์ พรีเมียร์ สุขุมวิท
🏠 OTA ที่อยู่: 12 ซ.สุขุมวิท 7, เขตวัฒนา, กรุงเทพฯ, 10110

เพื่อนบ้านของ OTA นี้ (Top-5 โรงแรมที่ใกล้ที่สุด):


,Hotel_Name,Hotel_Address,Hotel_Key,sim_key,sim_name,sim_addr,score
0,โรงแรมโนโวเทล ลิฟวิ่ง สุขุมวิท,12 ซ.สุขุมวิท 29(หลักเขตต์) แขวงคลองเตยเหนือ เ...,โนโวเทลลิฟวิ่งสุขุมวิท 12ซ.สุขุมวิท29(หลักเขตต...,0.863049,0.552809,0.442638,0.592823
1,โรงแรม เดอะคีย์ พรีเมียร์,12 ซ.สุขุมวิท 7(เลิศสิน 2) แขวงคลองเตยเหนือ เข...,เดอะคีย์พรีเมียร์ 12ซ.สุขุมวิท7(เลิศสิน2)แขวงค...,0.515356,0.521717,0.574103,0.530922
2,แทสโทเรีย คอลเลคชั่น โฮเทล สุขุมวิท,7 ซ.สุขุมวิท 20 แขวงคลองเตย เขตคลองเตย จ.กรุงเ...,แทสโทเรียคอลเลคชั่นสุขุมวิท 7ซ.สุขุมวิท20แขวงค...,0.284634,0.555203,0.522818,0.494612
3,โรงแรมคิวว์ (สุขุมวิท),9 ซ.สุขุมวิท 79(นิยม) แขวงพระโขนงเหนือ เขตวัฒน...,คิวว์สุขุมวิท 9ซ.สุขุมวิท79(นิยม)แขวงพระโขนงเห...,0.285542,0.575329,0.371313,0.476569
4,เฟรเซอร์ สวีท สุขุมวิท,38/8 ซ.สุขุมวิท 11 (ไชยยศ) แขวงคลองเตยเหนือ เข...,เฟรเซอร์สวีทสุขุมวิท 38/8ซ.สุขุมวิท11(ไชยยศ)แข...,0.432705,0.497468,0.444605,0.473943


In [69]:

# ---------------------------
# พารามิเตอร์
# ---------------------------
CAND_K = 50   # ผู้สมัครต่อ OTA จากคีย์ (แนะนำ 20–100 แล้วแต่ขนาดข้อมูล)
TOP_M  = 5    # จำนวนผลสุดท้ายต่อ OTA
WEIGHTS = dict(key=0.4, name=0.4, addr=0.2)
SCORE_THRESH = None  # เช่น 0.55 เพื่อกรองเฉพาะแมตช์ที่มั่นใจ

# ---------------------------
# เตรียมจำนวนแถว
# ---------------------------
n_trip  = len(OTA_df)
n_hotel = len(hotel)

# ---------------------------
# (ทางเลือก) ยืนยันว่า embeddings ถูก L2-normalize แล้ว
# ถ้ายัง ให้ normalize ก่อน (ป้องกัน dot != cosine)
# ---------------------------
# def _l2norm(x):
#     denom = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
#     return x / denom

# Uncomment ถ้าจำเป็น
# emb_trip_key  = _l2norm(emb_trip_key)
# emb_trip_name = _l2norm(emb_trip_name)
# emb_trip_addr = _l2norm(emb_trip_addr)
# emb_hotel_key  = _l2norm(emb_hotel_key)
# emb_hotel_name = _l2norm(emb_hotel_name)
# emb_hotel_addr = _l2norm(emb_hotel_addr)

# ---------------------------
# 1) คัดผู้สมัคร Top-K ด้วยคีย์ (เร็ว/สเกลดี)
# ---------------------------
nn = NearestNeighbors(n_neighbors=min(CAND_K, n_hotel), metric="cosine")
nn.fit(emb_hotel_key)
dist_k, idx_k = nn.kneighbors(emb_trip_key, return_distance=True)  # dist_k: (n_trip, CAND_K)
sim_key_k = 1.0 - dist_k                                           # cosine similarity

# ---------------------------
# 2) คำนวณ sim_name / sim_addr เฉพาะบนผู้สมัคร แล้วรวมคะแนน (vectorized)
#    รูปทรง:
#       emb_trip_name[:, None, :] -> (n_trip, 1, d)
#       emb_hotel_name[idx_k]     -> (n_trip, CAND_K, d)
# ---------------------------
sim_name_k = (emb_trip_name[:, None, :] * emb_hotel_name[idx_k]).sum(axis=2)  # (n_trip, CAND_K)
sim_addr_k = (emb_trip_addr[:, None, :] * emb_hotel_addr[idx_k]).sum(axis=2)  # (n_trip, CAND_K)

score_k = (WEIGHTS['key']  * sim_key_k +
           WEIGHTS['name'] * sim_name_k +
           WEIGHTS['addr'] * sim_addr_k)                                       # (n_trip, CAND_K)

# ---------------------------
# 3) เลือก Top-M ต่อ OTA (ไม่ใช่แค่ตัวชนะตัวเดียว)
# ---------------------------
# argsort จากมากไปน้อยต่อแถว
topm_order = np.argsort(-score_k, axis=1)[:, :TOP_M]                            # (n_trip, TOP_M)

top_idx   = idx_k[np.arange(n_trip)[:, None], topm_order]                       # (n_trip, TOP_M)
top_score = score_k[np.arange(n_trip)[:, None], topm_order]
top_skey  = sim_key_k[np.arange(n_trip)[:, None], topm_order]
top_sname = sim_name_k[np.arange(n_trip)[:, None], topm_order]
top_saddr = sim_addr_k[np.arange(n_trip)[:, None], topm_order]

# ---------------------------
# 4) สร้างผลลัพธ์แบบ "Top-M ต่อ OTA" (explode เป็นตารางยาว)
# ---------------------------
rows = []
for i in range(n_trip):
    for r in range(TOP_M):
        hidx = int(top_idx[i, r])
        rows.append({
            "OTA_Index": i,
            "OTA_Name": OTA_df.get("name", pd.Series([""]*n_trip))[i],
            "OTA_Address": OTA_df.get("address", pd.Series([""]*n_trip))[i],
            "Trip_Key": OTA_df.get("Trip_Key", pd.Series([""]*n_trip))[i],
            "Rank": r + 1,
            "Match_Hotel_Index": hidx,
            "Match_Hotel_Name": hotel.iloc[hidx]["Hotel_Name"],
            "Match_Hotel_Address": hotel.iloc[hidx]["Hotel_Address"],
            "Match_Hotel_Key": hotel.get("Hotel_Key", pd.Series([np.nan]*n_hotel)).iloc[hidx],
            "sim_key": float(top_skey[i, r]),
            "sim_name": float(top_sname[i, r]),
            "sim_addr": float(top_saddr[i, r]),
            "score": float(top_score[i, r]),
        })

df_topm = pd.DataFrame(rows)

# (ถ้ามี Hotel_Ref)
if "Hotel_Ref" in hotel.columns:
    df_topm["Match_Hotel_Ref"] = hotel["Hotel_Ref"].iloc[df_topm["Match_Hotel_Index"].values].values

# กรองด้วย threshold (ถ้าต้องการ)
if SCORE_THRESH is not None:
    df_topm_view = df_topm[df_topm["score"] >= SCORE_THRESH].reset_index(drop=True)
else:
    df_topm_view = df_topm.copy()

# ---------------------------
# 5) สร้างสรุป Top-1 ต่อ OTA สำหรับมุมมองรวม
# ---------------------------
df_top1 = (
    df_topm.sort_values(["OTA_Index", "score"], ascending=[True, False])
           .groupby("OTA_Index", as_index=False)
           .first()
           .reset_index(drop=True)
)

# ---------------------------
# 6) แสดงผล
# ---------------------------
topm_cols = [
    "OTA_Index","OTA_Name","OTA_Address","Trip_Key",
    "Rank","Match_Hotel_Name","Match_Hotel_Address","Match_Hotel_Key",
    "sim_key","sim_name","sim_addr","score"
]
display(df_topm_view[topm_cols].sort_values(["OTA_Index","Rank"]))

top1_cols = [
    "OTA_Index","OTA_Name","OTA_Address","Trip_Key",
    "Match_Hotel_Name","Match_Hotel_Address","Match_Hotel_Key",
    "sim_key","sim_name","sim_addr","score"
]
print("\n=== สรุป Top-1 ต่อ OTA ===")
display(df_top1[top1_cols].sort_values("score", ascending=False).reset_index(drop=True))


,OTA_Index,OTA_Name,OTA_Address,Trip_Key,Rank,Match_Hotel_Name,Match_Hotel_Address,Match_Hotel_Key,sim_key,sim_name,sim_addr,score
0,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,1,โรงแรมคิวว์ (สุขุมวิท),9 ซ.สุขุมวิท 79(นิยม) แขวงพระโขนงเหนือ เขตวัฒน...,คิวว์สุขุมวิท 9ซ.สุขุมวิท79(นิยม)แขวงพระโขนงเห...,0.665051,1.000000,0.667667,0.799554
1,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,2,โรงแรมอวานี สุขุมวิท กรุงเทพ,2089 ถ.สุขุมวิท แขวงพระโขนงเหนือ เขตวัฒนา จ.กร...,อวานีสุขุมวิท 2089สุขุมวิทแขวงพระโขนงเหนือเขตว...,0.630671,0.519823,0.459506,0.552099
2,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,3,โรงแรมคอส สุวรรณภูมิ แอร์พอร์ท,82 ถ.ร่มเกล้า แขวงมีนบุรี เขตมีนบุรี จ.กรุงเทพ...,คอสสุวรรณภูมิแอร์พอร์ท 82ร่มเกล้าแขวงมีนบุรีเข...,0.495924,0.655073,0.171052,0.494609
3,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,4,โซฟิเทล กรุงเทพ สุขุมวิท,189 ถ.สุขุมวิท แขวงคลองเตยเหนือ เขตวัฒนา จ.กรุ...,โซฟิเทลสุขุมวิท 189สุขุมวิทแขวงคลองเตยเหนือเขต...,0.451727,0.535342,0.426611,0.480150
4,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,5,ซีนิท สุขุมวิท กรุงเทพฯ,29/117 ซ.สุขุมวิท 3 ถ.สุขุมวิท แขวงคลองเตยเหนื...,ซีนิทสุขุมวิท 29/117ซ.สุขุมวิท3สุขุมวิทแขวงคลอ...,0.418852,0.568219,0.343104,0.463449
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,499,เดอะ วิคตอรี เอ็กเซกคิวทีฟ เรสซิเดนซ์,"37 ซอยศรีอยุธยา 12, ถนนศรีอยุธยา, เขตราชเทวี, ...",เดอะวิคตอรีเอ็กเซกคิวทีฟเรสซิเดนซ์ 37ซอยศรีอยุ...,1,โรงแรม แมริออท เอ็กเซ็กคิวทีฟ อพาร์ตเม้นต์ ทาว...,88/8 ซ.สุขุมวิท 49(กลาง) แขวงคลองตันเหนือ เขตว...,แมริออทเอ็กเซ็กคิวทีฟอพาร์ตเม้นต์ทาวน์ฮอล์ 88/...,0.527554,0.527554,0.140307,0.450105
2496,499,เดอะ วิคตอรี เอ็กเซกคิวทีฟ เรสซิเดนซ์,"37 ซอยศรีอยุธยา 12, ถนนศรีอยุธยา, เขตราชเทวี, ...",เดอะวิคตอรีเอ็กเซกคิวทีฟเรสซิเดนซ์ 37ซอยศรีอยุ...,2,แมริออท เอ็กเซ็กคิวทีฟ อพาร์ทเม้นต์ สุขุมวิ...,90 ซ.สุขุมวิท 24 แขวงคลองตัน เขตคลองเตย จ.กรุง...,แมริออทเอ็กเซ็กคิวทีฟอพาร์ทเม้นต์สุขุมวิทพาร์ค...,0.454866,0.454866,0.133888,0.390670
2497,499,เดอะ วิคตอรี เอ็กเซกคิวทีฟ เรสซิเดนซ์,"37 ซอยศรีอยุธยา 12, ถนนศรีอยุธยา, เขตราชเทวี, ...",เดอะวิคตอรีเอ็กเซกคิวทีฟเรสซิเดนซ์ 37ซอยศรีอยุ...,3,ลิฟวิ่ง แอ็ท ซิตี้ รีสอร์ท,161 ซ.สุขุมวิท39 แขวงคลองตันเหนือ เขตวัฒนา จ.ก...,ลิฟวิ่งแอ็ทซิตี้รีสอร์ท 161ซ.สุขุมวิท39แขวงคลอ...,0.375216,0.423574,0.300562,0.379628
2498,499,เดอะ วิคตอรี เอ็กเซกคิวทีฟ เรสซิเดนซ์,"37 ซอยศรีอยุธยา 12, ถนนศรีอยุธยา, เขตราชเทวี, ...",เดอะวิคตอรีเอ็กเซกคิวทีฟเรสซิเดนซ์ 37ซอยศรีอยุ...,4,อิสระ ซิตี้ โฮเต็ล,495 ซ.สารภี 12 ถ.อิสรภาพ แขวงสมเด็จเจ้าพระยา เ...,อิสระซิตี้โฮเต็ล 495ซ.สารภี12อิสรภาพแขวงสมเด็จ...,0.294573,0.470656,0.349835,0.376059



=== สรุป Top-1 ต่อ OTA ===


,OTA_Index,OTA_Name,OTA_Address,Trip_Key,Match_Hotel_Name,Match_Hotel_Address,Match_Hotel_Key,sim_key,sim_name,sim_addr,score
0,66,โรงแรมเซินเจิ้น ทาวเวอร์ กรุงเทพฯ,"3390 ถนน เพชรบุรีตัดใหม่ แขวงบางกะปิ, เขตห้วยข...",เซินเจิ้นทาวเวอร์ 3390เพชรบุรีตัดใหม่แขวงบางกะ...,โรงแรม เซินเจิ้น ทาวเวอร์,3390 ถ.เพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง...,เซินเจิ้นทาวเวอร์ 3390เพชรบุรีตัดใหม่แขวงบางกะ...,1.000000,1.000000,1.000000,1.000000
1,192,โรงแรมซีซั่น เพลส,"61/15 ถ.เพชรเกษม แขวงวัดท่าพระ, เขตบางกอกใหญ่,...",ซีซั่นเพลส 61/15เพชรเกษมแขวงวัดท่าพระเขตบางกอก...,โรงแรมซีซั่น เพลส,61/15 ถ.เพชรเกษม แขวงวัดท่าพระ เขตบางกอกใหญ่ จ...,ซีซั่นเพลส 61/15เพชรเกษมแขวงวัดท่าพระเขตบางกอก...,1.000000,1.000000,1.000000,1.000000
2,131,ม็อกซี่ แบงคอก ราชประสงค์,111 ถ. ราชดำริห์ แขวงลุมพินี เขตปทุมวัน กรุงเท...,ม็อกซี่แบงคอกราชประสงค์ 111ราชดําริห์แขวงลุมพิ...,โรงแรม ม็อกซี่ แบงคอก ราชประสงค์,111 ถ.ราชดำริห์ แขวงลุมพินี เขตปทุมวัน จ.กรุงเ...,ม็อกซี่แบงคอกราชประสงค์ 111ราชดําริห์แขวงลุมพิ...,1.000000,1.000000,1.000000,1.000000
3,83,ชามา เย็นอากาศ กรุงเทพ,"69 ถ. เย็นอากาศ แขวงช่องนนทรี, เขตยานนาวา, กรุ...",ชามาเย็นอากาศ 69เย็นอากาศแขวงช่องนนทรีเขตยานนา...,ชามา เย็นอากาศ กรุงเทพฯ,69 ถ.เย็นอากาศ แขวงช่องนนทรี เขตยานนาวา จ.กรุง...,ชามาเย็นอากาศ 69เย็นอากาศแขวงช่องนนทรีเขตยานนา...,1.000000,1.000000,1.000000,1.000000
4,144,โรงแรมซิทาดีนส์ สุขุมวิท 16 กรุงเทพ,"38 ซ. สุขุมวิท 16 แขวงคลองเตย, เขตคลองเตย, กรุ...",ซิทาดีนส์สุขุมวิท16 38ซ.สุขุมวิท16แขวงคลองเตยเ...,ซิทาดีนส์ สุขุมวิท 16,38 ซ.สุขุมวิท 16 แขวงคลองเตย เขตคลองเตย จ.กรุง...,ซิทาดีนส์สุขุมวิท16 38ซ.สุขุมวิท16แขวงคลองเตยเ...,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
495,274,โคโคเทล แบงค็อก เทวา ทองหล่อ,"2435 ถ. เพชรบุรีตัดใหม่ แขวงบางกะปิ, เขตห้วยขว...",โคโคเทลแบงค็อกเทวาทองหล่อ 2435เพชรบุรีตัดใหม่แ...,เลอ บองเนอ พอชเทล,"1575/33,1575/34 ถ.พหลโยธิน แขวงพญาไท เขตพญาไท ...",เลอบองเนอพอชเทล 1575/331575/34พหลโยธินแขวงพญาไ...,0.382672,0.311675,0.182829,0.314305
496,458,วินด์แฮม แบงค็อก ศูนย์ประชุมแห่งชาติสิริกิติ์,"388 ซ.ริมคลองไผ่สิงห์โต, ถ.พระราม 4, เขตคลองเต...",วินด์แฮมแบงค็อกศูนย์ประชุมแห่งชาติสิริกิติ์ 38...,แกรนด์ เซนเตอร์ พอยต์ เทอร์มินอล 21,2 ซ.สุขุมวิท 19 (วัฒนา) แขวงคลองเตยเหนือ เขตวั...,แกรนด์เซนเตอร์พอยต์เทอร์มินอล21 2ซ.สุขุมวิท19(...,0.355902,0.303594,0.215095,0.306817
497,421,อวาแลน พญาไท airport link/BTS,AVALAN ซอยติดรางรถไฟ ผ่านรางรถไฟแล้วเลี้ยวซ้าย...,อวาแลนพญาไทairportlinkbts avalanซอยติดรางรถไฟผ...,โรงแรม แอทนเรศ,213 ตรอกพุทธโอสถ แขวงสี่พระยา เขตบางรัก จ.กรุง...,แอทนเรศ 213ตรอกพุทธโอสถแขวงสี่พระยาเขตบางรักกร...,0.307355,0.302209,0.293942,0.302614
498,459,ราล์ฟ พอชเทล ประตูน้ำ,"131/19 ซอย ราชปรารภ 12, เขตราชเทวี, กรุงเทพฯ, ...",ราล์ฟพอชเทลประตูน้ํา 131/19ซอยราชปรารภ12เขตราช...,เกรซ,12 ซ.สุขุมวิท 3 ถ.สุขุมวิท แขวงคลองเตยเหนือ เข...,เกรซ 12ซ.สุขุมวิท3สุขุมวิทแขวงคลองเตยเหนือเขตว...,0.337761,0.247810,0.325391,0.299307


In [42]:
t_low  = 0.49
t_high = 0.65

def label_score(x):
    if x < t_low:
        return "not_match"
    elif x < t_high:
        return "possible"
    else:
        return "match"

df_top1["label"] = df_top1["score"].apply(label_score)


In [46]:
df_top1[['OTA_Name','Trip_Key','Match_Hotel_Name','Match_Hotel_Key', 'score']].drop_duplicates().sort_values('score', ascending=False).to_excel('out1.xlsx')

In [45]:
df_top1.label.value_counts()

,count
label,
match,207
possible,156
not_match,137


In [70]:
df_top1

,OTA_Index,OTA_Name,OTA_Address,Trip_Key,Rank,Match_Hotel_Index,Match_Hotel_Name,Match_Hotel_Address,Match_Hotel_Key,sim_key,sim_name,sim_addr,score,Match_Hotel_Ref
0,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,1,992,โรงแรมคิวว์ (สุขุมวิท),9 ซ.สุขุมวิท 79(นิยม) แขวงพระโขนงเหนือ เขตวัฒน...,คิวว์สุขุมวิท 9ซ.สุขุมวิท79(นิยม)แขวงพระโขนงเห...,0.665051,1.000000,0.667667,0.799554,135/2568
1,1,โรงแรมสำราญเพลส,"302 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",สําราญเพลส 302เขตราชเทวีกรุงเทพมหานคร10400,1,247,สำราญเพลส โฮเทล,302 ถ.เพชรบุรี แขวงถนนเพชรบุรี เขตราชเทวี จ.กร...,สําราญเพลส 302เพชรบุรีแขวงถนนเพชรบุรีเขตราชเทว...,0.586277,1.000000,0.618365,0.758184,204/2564
2,2,ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,"27 ซ.สุขุมวิท 4 คลองเตย, เขตคลองเตย, กรุงเทพฯ,...",ไอบิสสไตล์สุขุมวิท4 27ซ.สุขุมวิท4คลองเตยเขตคลอ...,1,607,โรงแรมโนโวเทล ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,27 ซ.สุขุมวิท 4 แขวงคลองเตย เขตคลองเตย จ.กรุงเ...,โนโวเทลไอบิสสไตล์สุขุมวิท4 27ซ.สุขุมวิท4แขวงคล...,0.725824,0.832416,0.712984,0.765893,175/2566
3,3,ทู ทรี อะ โฮมลี่ โฮเทล,"100 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ, เขตวัฒนา,...",ทูทรีอะโฮมลี่ 100ซ.สุขุมวิท23แขวงคลองเตยเหนือเ...,1,455,โรงแรม ทูทรี,100 ซ.สุขุมวิท 23(ประสานมิตร) แขวงคลองเตยเหนือ...,ทูทรี 100ซ.สุขุมวิท23(ประสานมิตร)แขวงคลองเตยเห...,0.662640,0.567245,0.569458,0.605845,148/2565
4,4,โรงแรมกลับ,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",กลับ 22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400,1,450,กลับโฮเทล,22/1 ซ.สมประสงค์ 2 แขวงถนนพญาไท เขตราชเทวี จ.ก...,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...,0.418987,1.000000,0.430295,0.653654,141/2565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,ณ ตะนาว 1969,"323 ถ.ตะนาว เสาชิงช้า, เขตพระนคร, กรุงเทพฯ, 10200",ณตะนาว1969 323ตะนาวเสาชิงช้าเขตพระนครกรุงเทพมห...,1,1013,โรงแรม แซดบี 69,551/1 ซ.พระรามที่ 2 ซอย 69 แขวงแสมดำ เขตบางขุน...,แซดบี69 551/1ซ.พระรามที่2ซอย69แขวงแสมดําเขตบาง...,0.303714,0.555735,0.089754,0.361730,166/2568
496,496,อะบลูม เอ็กซ์คลูซีฟ เซอร์วิส อพาร์ทเมนท์,"5/36 ซ.พหลโยธิน 3, เขตพญาไท, กรุงเทพฯ, 10400",อะบลูมเอ็กซ์คลูซีฟเซอร์วิสอพาร์ทเมนท์ 5/36ซ.พห...,1,454,โรงแรม ซัมเมอร์เซ็ท เอกมัย แบงค็อก - บาลานซ์,22 ซ.พาสนา 1 แขวงพระโขนงเหนือ เขตวัฒนา จ.กรุงเ...,ซัมเมอร์เซ็ทเอกมัยแบงค็อกบาลานซ์ 22ซ.พาสนา1แขว...,0.321412,0.564108,0.243099,0.402828,147/2565
497,497,โรงแรมเซ็นทารา วอเตอร์เกท พาวิลเลียน กรุงเทพฯ,"567 ถ.ราชปรารภ มักกะสัน, เขตราชเทวี, กรุงเทพฯ,...",เซ็นทาราวอเตอร์เกทพาวิลเลียน 567ราชปรารภมักกะส...,1,1262,เซ็นทาราวอเตอร์เกตพาวิเลียน กรุงเทพฯ,567 ถ.ราชปรารภ แขวงมักกะสัน เขตราชเทวี จ.กรุงเ...,เซ็นทาราวอเตอร์เกตพาวิเลียน 567ราชปรารภแขวงมัก...,0.770326,0.951349,0.717663,0.832203,119/2566
498,498,มาที่นี่โฮสเทล,"6 31-32 แขวงพญาไท, เขตพญาไท, กรุงเทพฯ, 10400",มาที่นี่โฮสเทล 631-32แขวงพญาไทเขตพญาไทกรุงเทพม...,1,1358,มิโด โฮเต็ล,222 ถ.ประดิพัทธ์ แขวงพญาไท เขตพญาไท จ.กรุงเทพม...,มิโดโฮเต็ล 222ประดิพัทธ์แขวงพญาไทเขตพญาไทกรุงเ...,0.449549,0.521164,0.429300,0.474145,226/2568


# TaskL LLM-confirmed
Use the Typhoon 4B model to compare the `Trip_Key` and `Match_Hotel_Key` columns in the `df_top1` DataFrame and store the boolean result in a new column called `LLM_confirmed`.

In [71]:
%pip install transformers

In [82]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "scb10x/typhoon2.1-gemma3-12b"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)
except Exception as e:
    print(f"Failed to load model with authentication: {e}")
    alternative_model_name = "airesearch/wangchanberta-base-att-spm-uncased"
    try:
        tokenizer = AutoTokenizer.from_pretrained(alternative_model_name)
        model = AutoModelForCausalLM.from_pretrained(alternative_model_name)
        model_name = alternative_model_name
    except Exception as e_alt:
        print(f"Failed to load alternative model: {e_alt}")
        model = None
        tokenizer = None

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Failed to load model with authentication: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `hf auth login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.


If you want to use `CamembertLMHeadModel` as a standalone, add `is_decoder=True.`


In [89]:
def confirm_match(trip_key: str, match_hotel_key: str) -> bool:
    if model is None or tokenizer is None:
        return False
    prompt = (
    f"Entity 1: {trip_key}\n"
    f"Entity 2: {match_hotel_key}\n"
    f"Are these the same place? ตอบเฉพาะ True หรือ False เท่านั้น:"
    )
    try:
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=10)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_lower = response.lower()
        if "true" in response_lower:
            return True
        elif "false" in response_lower:
            return False
        else:
            return False
    except Exception as e:
        print(f"Error during LLM inference: {e}")
        return False

In [90]:
tqdm.pandas()
df_top1['LLM_confirmed'] = df_top1.progress_apply(lambda row: confirm_match(row['Trip_Key'], row['Match_Hotel_Key']), axis=1)

  0%|          | 0/500 [00:00<?, ?it/s]

In [93]:
df_top1['LLM_confirmed'].value_counts()

,count
LLM_confirmed,
False,500


In [94]:
df_top1

,OTA_Index,OTA_Name,OTA_Address,Trip_Key,Rank,Match_Hotel_Index,Match_Hotel_Name,Match_Hotel_Address,Match_Hotel_Key,sim_key,sim_name,sim_addr,score,Match_Hotel_Ref,LLM_confirmed
0,0,โรงแรมคิวว์ สุขุมวิท,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,1,992,โรงแรมคิวว์ (สุขุมวิท),9 ซ.สุขุมวิท 79(นิยม) แขวงพระโขนงเหนือ เขตวัฒน...,คิวว์สุขุมวิท 9ซ.สุขุมวิท79(นิยม)แขวงพระโขนงเห...,0.665051,1.000000,0.667667,0.799554,135/2568,False
1,1,โรงแรมสำราญเพลส,"302 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",สําราญเพลส 302เขตราชเทวีกรุงเทพมหานคร10400,1,247,สำราญเพลส โฮเทล,302 ถ.เพชรบุรี แขวงถนนเพชรบุรี เขตราชเทวี จ.กร...,สําราญเพลส 302เพชรบุรีแขวงถนนเพชรบุรีเขตราชเทว...,0.586277,1.000000,0.618365,0.758184,204/2564,False
2,2,ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,"27 ซ.สุขุมวิท 4 คลองเตย, เขตคลองเตย, กรุงเทพฯ,...",ไอบิสสไตล์สุขุมวิท4 27ซ.สุขุมวิท4คลองเตยเขตคลอ...,1,607,โรงแรมโนโวเทล ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,27 ซ.สุขุมวิท 4 แขวงคลองเตย เขตคลองเตย จ.กรุงเ...,โนโวเทลไอบิสสไตล์สุขุมวิท4 27ซ.สุขุมวิท4แขวงคล...,0.725824,0.832416,0.712984,0.765893,175/2566,False
3,3,ทู ทรี อะ โฮมลี่ โฮเทล,"100 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ, เขตวัฒนา,...",ทูทรีอะโฮมลี่ 100ซ.สุขุมวิท23แขวงคลองเตยเหนือเ...,1,455,โรงแรม ทูทรี,100 ซ.สุขุมวิท 23(ประสานมิตร) แขวงคลองเตยเหนือ...,ทูทรี 100ซ.สุขุมวิท23(ประสานมิตร)แขวงคลองเตยเห...,0.662640,0.567245,0.569458,0.605845,148/2565,False
4,4,โรงแรมกลับ,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",กลับ 22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400,1,450,กลับโฮเทล,22/1 ซ.สมประสงค์ 2 แขวงถนนพญาไท เขตราชเทวี จ.ก...,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...,0.418987,1.000000,0.430295,0.653654,141/2565,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,ณ ตะนาว 1969,"323 ถ.ตะนาว เสาชิงช้า, เขตพระนคร, กรุงเทพฯ, 10200",ณตะนาว1969 323ตะนาวเสาชิงช้าเขตพระนครกรุงเทพมห...,1,1013,โรงแรม แซดบี 69,551/1 ซ.พระรามที่ 2 ซอย 69 แขวงแสมดำ เขตบางขุน...,แซดบี69 551/1ซ.พระรามที่2ซอย69แขวงแสมดําเขตบาง...,0.303714,0.555735,0.089754,0.361730,166/2568,False
496,496,อะบลูม เอ็กซ์คลูซีฟ เซอร์วิส อพาร์ทเมนท์,"5/36 ซ.พหลโยธิน 3, เขตพญาไท, กรุงเทพฯ, 10400",อะบลูมเอ็กซ์คลูซีฟเซอร์วิสอพาร์ทเมนท์ 5/36ซ.พห...,1,454,โรงแรม ซัมเมอร์เซ็ท เอกมัย แบงค็อก - บาลานซ์,22 ซ.พาสนา 1 แขวงพระโขนงเหนือ เขตวัฒนา จ.กรุงเ...,ซัมเมอร์เซ็ทเอกมัยแบงค็อกบาลานซ์ 22ซ.พาสนา1แขว...,0.321412,0.564108,0.243099,0.402828,147/2565,False
497,497,โรงแรมเซ็นทารา วอเตอร์เกท พาวิลเลียน กรุงเทพฯ,"567 ถ.ราชปรารภ มักกะสัน, เขตราชเทวี, กรุงเทพฯ,...",เซ็นทาราวอเตอร์เกทพาวิลเลียน 567ราชปรารภมักกะส...,1,1262,เซ็นทาราวอเตอร์เกตพาวิเลียน กรุงเทพฯ,567 ถ.ราชปรารภ แขวงมักกะสัน เขตราชเทวี จ.กรุงเ...,เซ็นทาราวอเตอร์เกตพาวิเลียน 567ราชปรารภแขวงมัก...,0.770326,0.951349,0.717663,0.832203,119/2566,False
498,498,มาที่นี่โฮสเทล,"6 31-32 แขวงพญาไท, เขตพญาไท, กรุงเทพฯ, 10400",มาที่นี่โฮสเทล 631-32แขวงพญาไทเขตพญาไทกรุงเทพม...,1,1358,มิโด โฮเต็ล,222 ถ.ประดิพัทธ์ แขวงพญาไท เขตพญาไท จ.กรุงเทพม...,มิโดโฮเต็ล 222ประดิพัทธ์แขวงพญาไทเขตพญาไทกรุงเ...,0.449549,0.521164,0.429300,0.474145,226/2568,False
